# Consumption and Generation Reference System

This tutorial is about the effect of the sign of active and reactive power in load and sgen in pandapower. In this tutorial, the load element is used to model power consumption and the static generator element is used to model power generation. In pandapower, the convention for consumption and generation is different. This difference between consumer frame convention and generator frame convention will be demonstrated here.

![reference_system](pics/reference_system.png)


![](l.png)

In regard to reactive power, there are 2 states for the 4 quadrants of each frame convention: under-excited and over-excited (see figure above).

In the Consumer Frame Convention, which is used for load-like elements (load, shunt, ward, xward, storage), the term ‘under-excited’ is used to indicate that the load is in I and II quadrants and is absorbing reactive power (decreases voltage) and the term ‘over-excited’ is used to indicate that the load is in the III and IV quadrants and is injecting reactive power (increases voltage).

In the Generator Frame Convention, which is used for generator-like elements (gen, sgen, ext_grid), the term ‘over-excited’ is used to indicate the generator is in the I and II quadrants and is injecting reactive power (increases voltage), and the term ‘under-excited’ is used to indicate the generator is in III and IV quadrants and is absorbing reactive power (decreases voltage).

We show the differences between the reference systems with the following example.

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_line,
    create_load,
    create_ext_grid,
    create_sgen
)
from pandapower.run import runpp
import pandas as pd
import seaborn as sn

In [ ]:
net=create_empty_network()
bus0 = create_bus(net, name="Bus 0", vn_kv=110, type="b")
bus1 = create_bus(net, name="Bus 1", vn_kv=110, type="b")
line0 = create_line(net, bus0, bus1, length_km=50, std_type="70-AL1/11-ST1A 110.0",  name="Line 1")
create_sgen(net, bus1, p_mw=0, q_mvar=0, name="static generator")
create_ext_grid(net, bus0, vm_pu=1.02, va_degree=50)
create_load(net, bus1, p_mw=0, q_mvar=0, name="load i")
net

In [ ]:
df=pd.DataFrame(columns=['element', 'p_mw', 'q_mvar', 'vm_pu', 'p_from_mw', 'p_to_mw'])

We will take three variations of p_mw (0, 50, -50) and three variations of q_mvar (0, 25, -25) for both load and sgen to analyze the output of the voltages.

In [ ]:
i=0
for element in ('load', 'sgen'):
    for x in [0, 50, -50]:
        for y in [0, 25, -25]:
            net[element].loc[0,['p_mw', 'q_mvar']] = x, y
            runpp(net)
            df.loc[i, 'element'] = element
            df.loc[i, ['p_mw', 'q_mvar']] = x,y
            df.loc[i, 'vm_pu'] = net.res_bus.vm_pu.at[bus1]
            df.loc[i, 'p_from_mw'] = net.res_line.p_from_mw.at[line0]
            df.loc[i, 'p_to_mw'] = net.res_line.p_to_mw.at[line0]
            i += 1
    net[element].loc[0, ['p_mw', 'q_mvar']] = 0, 0

In [ ]:
df

If you look at the 4th index, you will see both the active and reactive power of the load are positive and the voltage has been reduced. This is a result of consumption of P and Q in the under-excited state. In the 5th index the active power is positive but the reactive power is negative and as the result, the voltage does not drop as much as in the case of index 4.

Now we will see the plots for both the elements where the reference systems can be compared and the vm_pu values are shown for different combinations of active and reactive power.

Notice that, in the plot for the load element, when the sign of reactive power is negative, the reactive power is injected, which increases voltage. And when the reactive power is positive, the voltage drops because the reactive power is absorbed. The opposite effect can be seen for sgen elements.

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
g=sn.relplot(x="p_mw", 
        y="q_mvar",
        size="vm_pu",
        col="element",
        sizes=(500,500),
        palette="viridis",
        hue="vm_pu",
        data=df, alpha=1, zorder=10)
g.set(xlim=(-60,60),ylim=(-60,60))

texts =[['Under-excited (consumes reactive power, decreases voltage)', 'Over-excited (injects reactive power, increases voltage)'],
        ['Over-excited (injects reactive power, increases voltage)', 'Under-excited (consumes reactive power, decreases voltage)']]
        
for i,(ax, text) in enumerate(zip(g.axes[0], texts)):
    # Move left y-axis and bottom x-axis to centre, passing through (0,0)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    
    ax.xaxis.labelpad = 160
    ax.yaxis.labelpad = 160
    ax.grid()
    ax.text(0,50, text[0], bbox={'facecolor': 'lightgrey', 'pad': 10}, horizontalalignment='center', zorder=10)
    ax.text(0,-50, text[1], bbox={'facecolor': 'lightgrey', 'pad': 10}, horizontalalignment='center', zorder=10)
    
    rect=mpatches.Rectangle((0,-25),50,50, alpha=0.1,facecolor="blue")
    ax.add_patch(rect)
    

    # add text with text() function in matplotlib
    ax.text(10, 10,['P consumption','P generation'][i],fontsize=12)


   
